In [ ]:
from sklearn import neighbors
from sklearn.decomposition import PCA
from arch.bootstrap import StationaryBootstrap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math
import sys
import random

In [2]:
#read training and cross validation set
total = pd.read_csv("time_space.csv")
n = total.shape[0]

total = total.drop(total.columns[0], axis = 1) #delete index

tract_id = total['GEOID']
total = total.drop(total.columns[0], axis = 1) #delete tract_geoid
    
mp_date = total['MP_DATE']
origin = dt.datetime.strptime(total.loc[0, 'MP_DATE'],'%Y-%m-%d').date()
for i in range(n):
    now = dt.datetime.strptime(total.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    total.loc[i, 'MP_DATE'] = (now - origin).days/30.0

values = total['values']
total = total.drop(total.columns[3], axis = 1)

In [93]:
#get m block bootstrapping samples
s = 20
for j in range(1758):
    sub_data = total.iloc[63*j:63*(j+1)]
    bs = StationaryBootstrap(12, sub_data)
    if j == 0:
        train_lst = [data[0][0] for data in bs.bootstrap(s)]
    else:
        i = 0
        for data in bs.bootstrap(s):
            train_lst[i] = train_lst[i].append(data[0][0])
            i += 1

In [95]:
#get unselected samples as my cross validation
ind_all = set(total.index)
ind_diff = []
for i in range(s):
    ind_selected = set(train_lst[i].index)
    ind_diff.append(list(ind_all - ind_selected))
#create cross validation
cv_lst = [total.iloc[ind_diff[i]] for i in range(s)]

In [96]:
#find best k for each bootstrapping sample
j = 0
best_k_lst = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    cv_y = cv_lst[i]['dv']
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    cv_x = cv_lst[i].drop(cv_lst[i].columns[3], axis = 1)
    best_rss = sys.maxsize
    for j in range(2, 30, 2):
        knn = neighbors.KNeighborsRegressor(j, weights ='distance')
        y_ = knn.fit(train_x, train_y).predict(cv_x)
        mid = np.mean((y_ - cv_y)**2)
        if mid < best_rss:
            best_rss = mid
            best_k = j
        print((i, j))
    best_k_lst.append(best_k)

(0, 2)
(0, 4)
(0, 6)
(0, 8)
(0, 10)
(0, 12)
(0, 14)
(0, 16)
(0, 18)
(0, 20)
(0, 22)
(0, 24)
(0, 26)
(0, 28)
(1, 2)
(1, 4)
(1, 6)
(1, 8)
(1, 10)
(1, 12)
(1, 14)
(1, 16)
(1, 18)
(1, 20)
(1, 22)
(1, 24)
(1, 26)
(1, 28)
(2, 2)
(2, 4)
(2, 6)
(2, 8)
(2, 10)
(2, 12)
(2, 14)
(2, 16)
(2, 18)
(2, 20)
(2, 22)
(2, 24)
(2, 26)
(2, 28)
(3, 2)
(3, 4)
(3, 6)
(3, 8)
(3, 10)
(3, 12)
(3, 14)
(3, 16)
(3, 18)
(3, 20)
(3, 22)
(3, 24)
(3, 26)
(3, 28)
(4, 2)
(4, 4)
(4, 6)
(4, 8)
(4, 10)
(4, 12)
(4, 14)
(4, 16)
(4, 18)
(4, 20)
(4, 22)
(4, 24)
(4, 26)
(4, 28)
(5, 2)
(5, 4)
(5, 6)
(5, 8)
(5, 10)
(5, 12)
(5, 14)
(5, 16)
(5, 18)
(5, 20)
(5, 22)
(5, 24)
(5, 26)
(5, 28)
(6, 2)
(6, 4)
(6, 6)
(6, 8)
(6, 10)
(6, 12)
(6, 14)
(6, 16)
(6, 18)
(6, 20)
(6, 22)
(6, 24)
(6, 26)
(6, 28)
(7, 2)
(7, 4)
(7, 6)
(7, 8)
(7, 10)
(7, 12)
(7, 14)
(7, 16)
(7, 18)
(7, 20)
(7, 22)
(7, 24)
(7, 26)
(7, 28)
(8, 2)
(8, 4)
(8, 6)
(8, 8)
(8, 10)
(8, 12)
(8, 14)
(8, 16)
(8, 18)
(8, 20)
(8, 22)
(8, 24)
(8, 26)
(8, 28)
(9, 2)
(9, 4)
(9, 6)
(9, 8)


In [6]:
#get test data
test = pd.read_csv("test_data.csv")
m = test.shape[0]
test = test.drop(test.columns[0], axis = 1) #delete index
test_geoid = test['GEOID']
test = test.drop(test.columns[0], axis = 1) #delete GEOID
for i in range(m):
    now = dt.datetime.strptime(test.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    test.loc[i, 'MP_DATE'] = (now - origin).days/30.0

In [102]:
#get predictions
y_pre_lst = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_pre = knn.fit(train_x, train_y).predict(test)
    y_pre_lst.append(y_pre)

In [103]:
#get average predictions based on 10 predictions
y_pre_mat = np.array(y_pre_lst)
y_final_pre = np.mean(y_pre_mat, 0)

In [9]:
# draw graphs to show predictions of each tract
for i in range(1758):
    filename = "figure/%d.png" % i
    plt.figure(figsize=(8,6), dpi=60)
    plt.plot(range(12), y_final_pre[i*12:(i+1)*12], 'k')
    plt.savefig(filename, dpi = 60)

C:\Users\green\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [104]:
# final recommendations
recommend = dict()
recommend['invest'] = []
recommend['hold'] = []
recommend['divest'] = []
unique_tract = list((set(tract_id)))
tract_rate = []
for i in range(1758):
    future_rate = y_final_pre[i*12:(i+1)*12]
    tract_rate.append(np.mean(future_rate))
    if future_rate[1] < future_rate[0]:
        recommend['divest'].append(unique_tract[i])
    elif future_rate[0] > 0.2:
        recommend['invest'].append(unique_tract[i])
    else:
        recommend['hold'].append(unique_tract[i])

In [105]:
#output average increase rates for each tract
data = {'Future Annual Increase Rate': tract_rate,
        'TRACT_GEOID': unique_tract}
df = pd.DataFrame(data, index = range(1, 1759, 1))
df.to_csv('final_prediction.csv')

In [106]:
#output invest decisions for each tract
n_1 = len(recommend['divest'])
n_2 = len(recommend['hold'])
n_3 = len(recommend['invest'])
n_max = max(n_1, n_2, n_3)
for i in range(n_max - n_2):
    recommend['hold'].append(0)
for i in range(n_max - n_3):
    recommend['invest'].append(0)
df_2 = pd.DataFrame(recommend, index = range(1, n_max+1, 1))
df_2.to_csv('invest_decisions.csv')

In [107]:
#get model performance
MAE_lst = []
symbol_lst = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    cv_y = cv_lst[i]['dv']
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    cv_x = cv_lst[i].drop(cv_lst[i].columns[3], axis = 1)
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_ = knn.fit(train_x, train_y).predict(cv_x)
    MAE_lst.append(np.mean(abs(y_ - cv_y)))
    symbol_lst.append(sum(y_*cv_y>=0)/len(cv_y))

In [108]:
#draw MAE
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(s), MAE_lst, 'k')
plt.savefig("MAE.png", dpi = 60)
np.mean(MAE_lst)

5.443016068417621

In [109]:
#draw correct rate
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(s), symbol_lst, 'k')
plt.savefig("symbol.png", dpi = 60)
np.mean(symbol_lst)

0.84970478969517749